In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('attacks.csv', encoding='latin-1')
display(df.head())
print(df.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


(25723, 24)


<h3>Check NaN:</h3>

In [3]:
nan_columns = df.isnull().sum()
display(nan_columns[nan_columns > 0])
print(df.columns)

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')


<b>The columns "Unnamed: 22" and "Unnamed: 23" are empty and the columns "Age", "Time" and "Species" are more than 50% empty.</b>

In [4]:
colnames = []
for col in df.columns:
    colnames.append(col.lower())
    
colnames = [col.lower().replace(' ','').replace('/','_') for col in df.columns]
df.columns = colnames

df.drop(['unnamed:22','unnamed:23'], axis=1)
print(df.dtypes)

casenumber               object
date                     object
year                    float64
type                     object
country                  object
area                     object
location                 object
activity                 object
name                     object
sex                      object
age                      object
injury                   object
fatal(y_n)               object
time                     object
species                  object
investigatororsource     object
pdf                      object
hrefformula              object
href                     object
casenumber.1             object
casenumber.2             object
originalorder           float64
unnamed:22               object
unnamed:23               object
dtype: object


In [8]:
df_bycountry = df.country.value_counts()
df_bycountry = df_bycountry[df_bycountry >= 15]
display(df_bycountry)

USA                 2229
AUSTRALIA           1338
SOUTH AFRICA         579
PAPUA NEW GUINEA     134
NEW ZEALAND          128
BRAZIL               112
BAHAMAS              109
MEXICO                89
ITALY                 71
FIJI                  62
PHILIPPINES           61
REUNION               60
NEW CALEDONIA         53
CUBA                  46
MOZAMBIQUE            45
SPAIN                 44
INDIA                 40
EGYPT                 38
CROATIA               34
JAPAN                 34
PANAMA                32
SOLOMON ISLANDS       30
IRAN                  29
JAMAICA               27
FRENCH POLYNESIA      25
GREECE                25
HONG KONG             24
INDONESIA             23
ENGLAND               23
PACIFIC OCEAN         17
ATLANTIC OCEAN        17
COSTA RICA            17
BERMUDA               16
VIETNAM               15
TONGA                 15
Name: country, dtype: int64

In [15]:
list1 = df_bycountry.index.tolist()
list2 = []

for i in range(len(df.index)):
    if df.loc[i, 'country'] in list1:
        list2.append(df.loc[i, 'country'])
    else:
        list2.append('others')
        
df['country'] = list2
display(df)
print(df.country.unique())

,casenumber,date,year,type,country,area,location,activity,name,sex,...,investigatororsource,pdf,hrefformula,href,casenumber.1,casenumber.2,originalorder,unnamed:22,unnamed:23,Country
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN,USA
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN,USA
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN,USA
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN,AUSTRALIA
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN,MEXICO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,others,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
25719,NaN,NaN,NaN,NaN,others,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
25720,NaN,NaN,NaN,NaN,others,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others
25721,NaN,NaN,NaN,NaN,others,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,others


['USA' 'AUSTRALIA' 'MEXICO' 'BRAZIL' 'ENGLAND' 'SOUTH AFRICA' 'others'
 'COSTA RICA' 'BAHAMAS' 'NEW CALEDONIA' 'CUBA' 'NEW ZEALAND' 'SPAIN'
 'SOLOMON ISLANDS' 'JAPAN' 'EGYPT' 'REUNION' 'FRENCH POLYNESIA'
 'PHILIPPINES' 'INDONESIA' 'MOZAMBIQUE' 'FIJI' 'ITALY' 'ATLANTIC OCEAN'
 'GREECE' 'PAPUA NEW GUINEA' 'JAMAICA' 'TONGA' 'CROATIA' 'VIETNAM'
 'PANAMA' 'INDIA' 'HONG KONG' 'BERMUDA' 'IRAN' 'PACIFIC OCEAN']
